# Define constant variables

In [1]:
# Define the name of new folder
direct = "new"

# Define function
func = "1/math.exp(-x)"


In [2]:
# Import Library
import os
import shutil
import math
import random
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

In [3]:
# Create Directory/Folder
def MkDir(direct):
    os.mkdir("./" + direct)
    os.mkdir("./" + direct + "/images")
    os.mkdir("./" + direct + "/images" + "/train")
    os.mkdir("./" + direct + "/images" + "/val")
    os.mkdir("./" + direct + "/labels")
    os.mkdir("./" + direct + "/labels" + "/train")
    os.mkdir("./" + direct + "/labels" + "/val")
    
MkDir(direct)

# Find coefficient for train and val sets

In [4]:
class FindCoef:
    def __init__(self, set, func):
        self.set = set
        self.func = func
        self.a1 = [] # 0
        self.a2 = [] # 1
        self.a3 = [] # 2
        self.a4 = [] # 0_1
        self.a5 = [] # 0_2
        self.a6 = [] # 1_2
        self.a7 = [] # All
        self.d = {"0": 0, "1": 0, "2": 0, "0_1": 0, "0_2": 0, "1_2": 0, "All": 0}
        self.w = [0.3, 0.2, 0.5]
        
    
    def OpenF(self, folder):
        path = "./" + self.set + folder + "/images"
        files = os.listdir(path)
        A = [i.split('.')[0] for i in files]
        return A
        
        
    def findImg(self):
        self.a1 = self.OpenF("0")
        self.a2 = self.OpenF("1")
        self.a3 = self.OpenF("2")
        self.a4 = self.OpenF("0_1")
        self.a5 = self.OpenF("0_2")
        self.a6 = self.OpenF("1_2")
        self.a7 = self.OpenF("All")
        
        
    def count(self, A, folder):
        path = "./" + self.set + folder + "/labels/"
        check = [0, 0, 0]
        for i in A:
            f = open(path + i + ".txt")
            line = f.readlines()

            for line_ in line:
                u = line_.split(" ")
                if (u[0] == '0'):
                    check[0] += 1
                elif (u[0] == '1'):
                    check[1] += 1
                else:
                    check[2] += 1
            f.close()
        return check
    
    def f(self, x, func):
        return eval(func)

    def transform(self, w, func):
        A = list(map(lambda w_ : self.f(w_, func), w))
        A = list(map(lambda a : a / sum(A), A))
        A = [round(i, 2) for i in A]
        return A
    
    
    def Matrix(self):
        self.A = []
        self.A += [self.count(self.a1, "0")]
        self.A += [self.count(self.a2, "1")]
        self.A += [self.count(self.a3, "2")]
        self.A += [self.count(self.a4, "0_1")]
        self.A += [self.count(self.a5, "0_2")]
        self.A += [self.count(self.a6, "1_2")]
        self.A += [self.count(self.a7, "All")]
        self.A = np.array(self.A).T
        self.A_ = self.A[0:2,2:]
        self.p = self.transform(self.w, self.func)
        
        
    def isValid(self, X, minError, train=True):
        if not all([x > 0.5 for x in X.squeeze()]):
            return False, -1, -1
        maxError = max([abs(x-round(x)) for x in X.squeeze()])
        if maxError > minError:
            return False, -1, -1

        X = np.array([round(x) for x in X.squeeze()])
        numImg = int(sum(np.array(list(map(lambda a : len(a), [self.a1, self.a2, self.a3, self.a4, self.a5, self.a6, self.a7]))) * X))
        if (numImg >= 3000*(0.8 if train else 0.1) or numImg <= (2100 if train else 240)):
            return False, -1, -1

        return True, numImg, maxError
    
        
    def Coefficient(self):
        cnt = 0
        minError = 100000000000000000000000
        h = 20
        
        # fix X(3,4,5,6,7)
        x34567 = [np.array([0,0,x3,x4,x5,x6,x7], dtype=np.float32).reshape((7,1)) for x3 in range(1, h) for x4 in range(1, 10) for x5 in range(1, h) for x6 in range(1, 10) for x7 in range(1, h)]
        for X in x34567:
            r2 = np.dot(self.A, X)[2][0]
            # B * R = C
            B = np.array([[self.p[0]-1, self.p[0], self.p[0]],
                          [self.p[1], self.p[1]-1, self.p[1]],
                          [0,0,1]])
            C = np.array([[0],[0],[r2]])
            R = np.linalg.inv(B) @ C
            
            X12 = R[:-1,:] - (self.A_ @ X[2:,:])
            X[0], X[1] = X12[0] / self.A[0][0], X12[1] / self.A[1][1]
            
            if (self.set == "train/"):
                cond, numImg, maxError = self.isValid(X, minError)
            else:
                cond, numImg, maxError = self.isValid(X, minError, False)
                
            if cond:
                cnt += 1
                if maxError < minError:
                    minError = maxError
                    self.bestSolution = np.array([round(x) for x in X.squeeze()], dtype="int32")
                    self.bestNum = numImg

        print("count:", cnt)
        print("Images:", self.bestNum)
        print(self.bestSolution)
        
        
    def Save(self):
        k = 0
        for i in self.d.keys():
            self.d[i] = self.bestSolution[k]
            k += 1
    
    

In [5]:
# Find coefficient for train and val

print("train:")
train = FindCoef("train/", func)
train.findImg()
train.Matrix()
train.Coefficient()

print()

print("val:")
val = FindCoef("val/", func)
val.findImg()
val.Matrix()
val.Coefficient()

print("\n")
total = train.bestNum + val.bestNum
print("train: ", round(train.bestNum*100/total, 2), "%", sep="")
print("val: ", round(val.bestNum*100/total, 2), "%", sep="")

train:
count: 30
Images: 2395
[ 3  1 19  1  3  1  1]

val:
count: 51
Images: 276
[ 3  1 16  3  9  2  1]


train: 89.67%
val: 10.33%


In [6]:
# Make sure train/val = 80/20

val.bestSolution = val.bestSolution * math.ceil(2 / (val.bestNum * 10 / train.bestNum))
val.bestNum *= math.ceil(2 / (val.bestNum * 10 / train.bestNum))

train.bestSolution = train.bestSolution - 1
val.bestSolution = val.bestSolution - 1

train.Save()
val.Save()

print("Coef Train:", train.d)
print("Coef Val:", val.d)



Coef Train: {'0': 2, '1': 0, '2': 18, '0_1': 0, '0_2': 2, '1_2': 0, 'All': 0}
Coef Val: {'0': 5, '1': 1, '2': 31, '0_1': 5, '0_2': 17, '1_2': 3, 'All': 1}


# Augmentation

In [7]:
def aug(num, path, cat, seq, k):
    # image
    image = np.asarray(Image.open(path + '/images' + "/" + num + '.jpg'))
    H, W = image.shape[:2]
    # label
    bbs = []
    with open(path + '/labels' + "/" + num + '.txt', 'r') as file:
        for line_ in file:
            line = line_.split(' ')
            label = line[0]
            xc, yc, w, h = [float(x) for x in line[1:]]
            xc, yc, w, h = xc*W, yc*H, w*W, h*H

            bbs.append(BoundingBox(x1=xc-w/2, y1=yc-h/2, x2=xc+w/2, y2=yc+h/2, label=label))

    bbs = BoundingBoxesOnImage(bbs, shape=image.shape[:2])

    # Augment BBs and images.
    image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

    # clip off the outside label
    bbs_aug = bbs_aug.remove_out_of_image().clip_out_of_image()
        
    # save image and label
    # image with BBs before/after augmentation (shown below)
    Image.fromarray(image_aug).save("./" + direct + "/images" + cat + "/" + num + "_" + str(k) + '.jpg')
    
    # Copy
    oldtxt = path + '/labels/' + num + '.txt'
    newtxt = "./" + direct + "/labels" + cat + "/" + num + "_" + str(k) + '.txt'
    shutil.copy(oldtxt, newtxt)
    
    # New size of image
    H_, W_ = image_aug.shape[:2]
    
    # labels after aug
    with open("./" + direct + "/labels" + cat + "/" + num + "_" + str(k) + '.txt', 'w') as file:
        for bb in bbs_aug:
            x1, y1, x2, y2, label = bb.x1, bb.y1, bb.x2, bb.y2, bb.label
            xc = ((x1+x2) / 2) / W_
            yc = ((y1+y2) / 2) / H_
            w  = (x2-x1) / W_
            h  = (y2-y1) / H_
            data = ' '.join([str(a) for a in [label, xc, yc, w, h]])
            file.write(data + '\n')


In [8]:
seq1 = iaa.Sequential([  
    # shift
    iaa.Sometimes(
        0.3,
        iaa.KeepSizeByResize(iaa.Affine(
            translate_px={"x": (-100,100), "y": (-100,100)},
            fit_output = False
        ))
    ),
    
    # crop
    iaa.Sometimes(
        0.5,
        iaa.Crop(percent=([0.05, 0.1], [0.1, 0.2], [0.05, 0.1], [0.1, 0.2]),
             keep_size=True)
    )
    
    
], random_order=False) # apply augmenters in random order



seq2 = iaa.Sequential([
    seq1,
    
    # Strengthen or weaken the contrast in each image.
    iaa.Sometimes(
        0.5,
        iaa.LinearContrast((0.75, 1.25))
    ),

    # Add gaussian noise.
    iaa.Sometimes(
        0.3,
        iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)
    ),

    # Make some images brighter and some darker.
    iaa.Sometimes(
        0.5,
        iaa.Multiply((0.8, 1.3), per_channel=0.2)
    ),
    
    # But we only blur about 75% of all images.
    iaa.Sometimes(
        0.3,
        iaa.GaussianBlur(sigma=(0.0, 2))
    ),
    
    # Apply affine transformations to each image.
    # zoom
    iaa.Sometimes(
        0.5,
        iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
            fit_output = True
        )
    ),

    # cut out
    iaa.Sometimes(
        0.3,
        iaa.Cutout(nb_iterations=(5, 20), size=0.025, squared=True)
    ),

    # rotate
    iaa.Sometimes(
       0.5,
       iaa.Affine(
       rotate=(-90,90),
       fit_output = True
       )
    ),
    
    # flip
    iaa.Sometimes(
        0.5,
        iaa.flip.Fliplr(1)
    ),

], random_order=True) # apply augmenters in random order


seq3 = iaa.Sequential([  
    seq2,
    
    iaa.Sometimes(
        0.5,
        iaa.Resize((0.5, 1.0))
    ),
    
    
    
], random_order=False) # apply augmenters in random order

In [9]:
# Augment train set
for x, y in train.d.items():
    files = os.listdir("./train/" + x +"/images")
    for i in tqdm(files):
        num = i.split(".")[0]
        
        shutil.copy("./train/" + x + "/images/" + i, "./" + direct + "/images" + "/train")
        shutil.copy("./train/" + x + "/labels/" + num + ".txt", "./" + direct + "/labels" + "/train")
        
        for j in range(1, y+1):
            #seq = choose_seq(seq_)
            aug(num, "./train/" + x, "/train", seq2, j)
            
            path1 = "./" + direct + "/labels" + "/train" + "/" + num + "_" + str(j) + '.txt'
        


100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 305.46it/s]


In [10]:
# Augment val set
for x, y in val.d.items():
    files = os.listdir("./val/" + x +"/images")
    for i in tqdm(files):
        num = i.split(".")[0]
        
        shutil.copy("./val/" + x + "/images/" + i, "./" + direct + "/images" + "/val")
        shutil.copy("./val/" + x + "/labels/" + num + ".txt", "./" + direct + "/labels" + "/val")
        
        for j in range(1, y+1):
            #seq = choose_seq(seq_)
            aug(num, "./val/" + x, "/val", seq3, j)
        


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.18it/s]


In [11]:
# Check number of images and labels are correct
path = "./" + direct

print("Images train:", train.bestNum == len(os.listdir(path + "/images/train")))
print("Labels train:", train.bestNum == len(os.listdir(path + "/labels/train")))

print("Images val:", val.bestNum == len(os.listdir(path + "/images/val")))
print("Labels val:", val.bestNum == len(os.listdir(path + "/labels/val")))

Images train: True
Labels train: True
Images val: True
Labels val: True


# Shuffle, Remove, Bias, Zip

In [12]:
path = "./" + direct
imgs = os.listdir(path + "/images/train")
names = [img.split(".")[0] for img in imgs]

In [13]:
# If the size of folder surpasses 500MB, kindly uncomment the code below and run it

for name in tqdm(names):
    img = Image.open(path + "/images/train/" + name + ".jpg")
    img.save(path + "/images/train/" + name + "_0.jpg", optimize=True)
    os.remove(path + "/images/train/" + name + ".jpg")
    os.rename(path + "/images/train/" + name + "_0.jpg", path + "/images/train/" + name + ".jpg")
    img.close()

100%|██████████████████████████████████████████████████████████████████████████████| 2395/2395 [02:06<00:00, 18.96it/s]


In [14]:
# Choose random constants
a = random.randint(0, 10000000000)
b = random.randint(0, 10000000000)
m = random.randint(0, 10000000000)
x = random.randint(0, 10000000000)

print("a:", a)
print("b:", b)
print("m:", m)
print("x:", x)

a: 1295092351
b: 6054668326
m: 524924142
x: 4956792354


In [15]:
# Shuffle train set
for name in tqdm(names):
    x = (a*x+b) % m
    os.rename(path + "/images/train/" + name + ".jpg", path + "/images/train/" + str(x) + ".jpg")
    os.rename(path + "/labels/train/" + name + ".txt", path + "/labels/train/" + str(x) + ".txt")

100%|████████████████████████████████████████████████████████████████████████████| 2395/2395 [00:00<00:00, 3406.96it/s]


In [16]:
# Remove zero label images in train and val set
def findRemove(path, set, zero_label):
    path_ = path + "/labels/" + set
    li = os.listdir(path_)
    for i in li:
        path__ = path_ + i
        f = open(path__)
        line = f.readlines()
        if (len(line) == 0):
            zero_label.append(i.split(".")[0])
        f.close()
        
    for i in zero_label:
        os.remove(path + "/images/" + set + i + ".jpg")
        os.remove(path + "/labels/" + set + i + ".txt")


zero_train = []
zero_val = []

findRemove(path, "train/", zero_train)
findRemove(path, "val/", zero_val)


print("Remove", len(zero_train), "image(s) in train")
print("Remove", len(zero_val), "image(s) in val")

Remove 37 image(s) in train
Remove 21 image(s) in val


In [17]:
# Check Bias
def Bias(array, p):
    files = os.listdir(p)
    check = [0, 0, 0]

    for file in array:
        f = open(p + str(file) + ".txt")
        line = f.readlines()

        for line_ in line:
            u = line_.split(" ")
            if (u[0] == '0'):
                check[0] += 1
            elif (u[0] == '1'):
                check[1] += 1
            else:
                check[2] += 1
        f.close()
    return check
        
def result(check, sth=""):
    s = sum(check)
    print("Total", sth, "labels:", s)
    print('0: ', check[0], '\t\t', round(check[0]*100/s, 2), '%', sep="")
    print('1: ', check[1], '\t\t', round(check[1]*100/s, 2), '%', sep="")
    print('2: ', check[2], '\t\t', round(check[2]*100/s, 2), '%', sep="")
    print("\n")
    
    
path_ = path + "/labels/"

a_t = os.listdir(path_ + "train")
t = [i.split(".")[0] for i in a_t]
a_v = os.listdir(path_ + "val")
v = [i.split(".")[0] for i in a_v]

t_ = np.array(Bias(t[:], path_ + "train/"))
v_ = np.array(Bias(v[:], path_ + "val/"))

print("RESULT")
result(t_, 'train')
result(v_, 'val')

print("Expected")
print("0: ", round(train.p[0]*100, 2), "%", sep="")
print("1: ", round(train.p[1]*100, 2), "%", sep="")
print("2: ", round(train.p[2]*100, 2), "%", sep="")
print("\n")

# The result is a bit different from expected due to zero label images after augmentation

print("Images:")
t__ = len(os.listdir(path_ + "train/"))
v__ = len(os.listdir(path_ + "val/"))
print("train:", round(t__*100/(t__+v__), 2))
print("train:", round(v__*100/(t__+v__), 2))

RESULT
Total train labels: 3101
0: 903		29.12%
1: 1020		32.89%
2: 1178		37.99%


Total val labels: 727
0: 224		30.81%
1: 206		28.34%
2: 297		40.85%


Expected
0: 32.0%
1: 29.0%
2: 39.0%


Images:
train: 81.62
train: 18.38


In [18]:
# Make zip file
shutil.make_archive(direct, 'zip', "./", direct)

'D:\\Anaconda\\A_Python\\Competition\\Data-Centric-AI-Competition\\new.zip'